In [1]:
import numpy as np
from scipy.special import softmax

from transformers import Trainer
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification

from datasets import load_metric
from datasets import load_dataset
from datasets import load_from_disk

from sklearn.metrics import accuracy_score, recall_score, f1_score

from transformers import AutoTokenizer, DataCollatorWithPadding, BertForSequenceClassification

In [2]:
import torch
torch.cuda.empty_cache()

In [3]:
import os
print(os.getcwd())

d:\OneDrive\Cornell\Fall 2023\Analyzing-the-Correlation-Between-Retail-Traders--Sentiments-and-Equity-Market-Movements\Sentiment_Analysis


In [4]:
# !pip install datasets

In [5]:
# raw_datasets = load_from_disk("data")
raw_datasets = load_from_disk("data/finetune_data/")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["text"], padding=True, truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # pad all the examples to the length of the longest element when we batch elements together — dynamic padding.

In [6]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3628
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 908
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5656
    })
})

In [7]:
raw_datasets['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 3628
})

In [8]:
raw_datasets['train']['label']

[1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,


In [9]:
tokenized_datasets['train']['text']

['After the takeover , Cramo will become the second largest rental services provider in the Latvian market .',
 'This is the second successful effort for the financial investor this year .',
 "$SIRI is losing its signal. SiriusXM's satellite radio business faces streaming competition.",
 'Finnish Scanfil , a systems supplier and contract manufacturer to the communications sector and the electronics industry , reports its net sales totalled EUR 94.7 mn in the first half of 2010 , down from EUR 99.5 mn in the first half of 2009 .',
 'Amazon to attack UK grocery market with Morrisons deal',
 '$ISRG PT raised to $700 from $640 at Leerink - keeps Outperform rated',
 '$MARA faces profitability challenges as cryptocurrency markets become increasingly volatile, impacting its stock.',
 'Walmart is down over 10% today, on pace for its largest decline since October 1987. $WMT',
 'Pretax profit rose to EUR 1,019 mn from EUR 1,007 in the corresponding period in 2005 .',
 'Social Media stocks catchi

In [10]:
metric = load_metric("glue", "mrpc")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-10-093e3287862a>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "mrpc")


In [11]:
# !pip install transformers[torch]
# !pip install accelerate -U

In [12]:
# !pip install accelerate==0.24.0

In [13]:
import accelerate
print(accelerate.__version__)

0.24.1


In [14]:
training_args = TrainingArguments(
    output_dir="test-trainer",
    evaluation_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=256,
    # During the first 500 training steps, the learning rate gradually increases from 0 (or a small base rate) to the specified learning rate.
    # This gradual increase helps in stabilizing the training process and often leads to better performance, as it prevents the model from making too large updates too quickly.
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='logs',
    logging_steps=10,  # how frequently the training progress is logged
    save_strategy="epoch",  # Set save strategy to match evaluation strategy
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    # fp16=True,  # Enable mixed precision training
)
# model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
model = BertForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

c:\Users\Howard\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\modeling_utils.py:402: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(checkpoint_file, framework="pt") as f:
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializ

In [15]:
trainer.train()

c:\Users\Howard\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Howard/.netrc


0%|          | 0/1135 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.6881, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.04}
{'loss': 0.6796, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.09}
{'loss': 0.681, 'learning_rate': 3e-06, 'epoch': 0.13}
{'loss': 0.6536, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.18}
{'loss': 0.6558, 'learning_rate': 5e-06, 'epoch': 0.22}
{'loss': 0.6256, 'learning_rate': 6e-06, 'epoch': 0.26}
{'loss': 0.6207, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.31}
{'loss': 0.619, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.35}
{'loss': 0.5426, 'learning_rate': 9e-06, 'epoch': 0.4}
{'loss': 0.4953, 'learning_rate': 1e-05, 'epoch': 0.44}
{'loss': 0.4761, 'learning_rate': 1.1000000000000001e-05, 'epoch': 0.48}
{'loss': 0.4717, 'learning_rate': 1.2e-05, 'epoch': 0.53}
{'loss': 0.3871, 'learning_rate': 1.3000000000000001e-05, 'epoch': 0.57}
{'loss': 0.3655, 'learning_rate': 1.4000000000000001e-05, 'epoch': 0.62}
{'loss': 0.4174, 'learning_rate': 1.5e-05, 'epoch': 0.66}
{'loss': 0.3793, 'learning

0%|          | 0/4 [00:00<?, ?it/s]

...

0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.3392498791217804, 'eval_accuracy': 0.9328193832599119, 'eval_f1': 0.9415148609779482, 'eval_runtime': 3.7127, 'eval_samples_per_second': 244.563, 'eval_steps_per_second': 1.077, 'epoch': 4.0}
{'loss': 0.0021, 'learning_rate': 1.7716535433070868e-05, 'epoch': 4.01}
{'loss': 0.0028, 'learning_rate': 1.692913385826772e-05, 'epoch': 4.05}
{'loss': 0.0014, 'learning_rate': 1.6141732283464566e-05, 'epoch': 4.1}
{'loss': 0.0037, 'learning_rate': 1.535433070866142e-05, 'epoch': 4.14}
{'loss': 0.0012, 'learning_rate': 1.4566929133858267e-05, 'epoch': 4.19}
{'loss': 0.0421, 'learning_rate': 1.377952755905512e-05, 'epoch': 4.23}
{'loss': 0.045, 'learning_rate': 1.2992125984251968e-05, 'epoch': 4.27}
{'loss': 0.0009, 'learning_rate': 1.220472440944882e-05, 'epoch': 4.32}
{'loss': 0.0351, 'learning_rate': 1.141732283464567e-05, 'epoch': 4.36}
{'loss': 0.0011, 'learning_rate': 1.062992125984252e-05, 'epoch': 4.41}
{'loss': 0.0593, 'learning_rate': 9.842519685039371e-06, 'epoch': 4.45

0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.3353908956050873, 'eval_accuracy': 0.9438325991189427, 'eval_f1': 0.9518413597733711, 'eval_runtime': 3.6951, 'eval_samples_per_second': 245.73, 'eval_steps_per_second': 1.083, 'epoch': 5.0}
{'train_runtime': 428.7062, 'train_samples_per_second': 42.313, 'train_steps_per_second': 2.648, 'train_loss': 0.1730779605002721, 'epoch': 5.0}


TrainOutput(global_step=1135, training_loss=0.1730779605002721, metrics={'train_runtime': 428.7062, 'train_samples_per_second': 42.313, 'train_steps_per_second': 2.648, 'train_loss': 0.1730779605002721, 'epoch': 5.0})

In [16]:
predictions = trainer.predict(tokenized_datasets["test"])
print(predictions.predictions.shape, predictions.label_ids.shape)

0%|          | 0/23 [00:00<?, ?it/s]

(5656, 2) (5656,)


In [17]:
predictions

PredictionOutput(predictions=array([[-0.34344333,  0.63074565],
       [-1.8354884 ,  2.053616  ],
       [ 1.1662291 , -1.4730583 ],
       ...,
       [ 1.3780316 , -1.5695528 ],
       [-1.942197  ,  2.173619  ],
       [-1.9871747 ,  2.2119365 ]], dtype=float32), label_ids=array([1, 1, 1, ..., 1, 1, 1], dtype=int64), metrics={'test_loss': 0.6344084143638611, 'test_accuracy': 0.7446958981612447, 'test_f1': 0.8057573311810601, 'test_runtime': 10.5932, 'test_samples_per_second': 533.927, 'test_steps_per_second': 2.171})

In [18]:
# Apply softmax to convert logits to probabilities
probabilities = softmax(predictions.predictions, axis=1)

# Get the predicted class labels
predicted_labels = np.argmax(probabilities, axis=1)

print("Probabilities:\n", probabilities)
print("Predicted Labels:\n", predicted_labels)

Probabilities:
 [[0.27404633 0.72595364]
 [0.0200533  0.9799467 ]
 [0.9333477  0.06665236]
 ...
 [0.95014924 0.04985081]
 [0.01605079 0.98394924]
 [0.01478698 0.985213  ]]
Predicted Labels:
 [1 1 0 ... 0 1 1]


In [19]:
true_labels = tokenized_datasets["test"]["label"]

accuracy = accuracy_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

print("Accuracy:", accuracy)
print("Recall:", recall)
print("F1 Score:", f1)


Accuracy: 0.7446958981612447
Recall: 0.7446958981612447
F1 Score: 0.7420506375250072
